In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
import os
from tensorflow.keras import layers, models
from sklearn.metrics import confusion_matrix
from tensorflow.keras.datasets import mnist


In [4]:
# !unzip team13-20240417T085751Z-001.zip

In [5]:
def load_images_from_folder(folder):
    images = []
    labels = []
    class_names = os.listdir(folder)
    # print(class_names)
    class_index = 0

    for class_name in class_names:
        class_folder = os.path.join(folder, class_name)
        if os.path.isdir(class_folder):
            for filename in os.listdir(class_folder):
                img_path = os.path.join(class_folder, filename)
                # Read the image using OpenCV
                image = cv2.imread(img_path)
                # Resize the image if needed
                # image = cv2.resize(image, (width, height))
                if image is not None:
                    images.append(image)
                    labels.append(class_index)
        class_index += 1

    return np.array(images), np.array(labels)


In [6]:
train_images, train_labels = load_images_from_folder('task1/train')
test_images, test_labels = load_images_from_folder('task1/test')
valid_images, valid_labels = load_images_from_folder('task1/valid')

print('Train images:', len(train_images))
print('Test images:', len(test_images))
print('Valid images:', len(valid_images))

Train images: 2000
Test images: 500
Valid images: 500


In [7]:
# class index to name
# 0 - beetle
# 1 - sloth
# 2 - pencil pouch
# 3 - xerox machine
# 4 - vase

In [8]:
print(train_images[0].shape)
# print(train_images[0])
num_classes=5

(224, 224, 3)


In [9]:
train_images = train_images / 255.0
test_images = test_images / 255.0
valid_images = valid_images / 255.0

# print(train_images[0])

In [10]:
def create_model(num_cl2_feature_maps):
    model = models.Sequential()

    # Convolutional Layer 1
    model.add(layers.Conv2D(4, (3, 3), activation='relu', input_shape=(224, 224, 3)))
    # Pooling Layer 1
    model.add(layers.AveragePooling2D((2, 2), strides=(2, 2)))

    # Convolutional Layer 2
    model.add(layers.Conv2D(num_cl2_feature_maps, (3, 3), activation='relu'))
    # Pooling Layer 2
    model.add(layers.AveragePooling2D((2, 2), strides=(2, 2)))

    # Flatten the output of the previous layer
    model.add(layers.Flatten())

    # Fully connected layer
    model.add(layers.Dense(128, activation='relu'))
    # Output layer
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model



In [11]:
# Define the model with specified number of feature maps in CL2
# model = create_model(num_cl2_feature_maps=10)  # You can adjust the number of feature maps as per your requirement
#hypermater testing

def get_test_accuracy(hidden_units):

    model = create_model(hidden_units)
    # Compile the model
    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(train_images, train_labels, epochs=5, validation_data=(valid_images, valid_labels),verbose=0)

    _, test_accuracy = model.evaluate(test_images, test_labels)
    return test_accuracy

In [12]:
test_accuracy_list = []
hidden_units_list = [i for i in range(1, 11)]

for hidden_units in hidden_units_list:
    test_accuracy = get_test_accuracy(hidden_units)
    test_accuracy_list.append(test_accuracy)


#plot the graph
import matplotlib.pyplot as plt
plt.plot(hidden_units_list, test_accuracy_list)
plt.xlabel('Number of feature maps in CL2')


/Users/adityaviraj/Desktop/DL assgn2/.venv/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-05-25 00:29:17.086509: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-05-25 00:29:17.086563: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-05-25 00:29:17.086567: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-05-25 00:29:17.086598: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-05-25 00:29:17.086612: I tensorflow/core/common_runtime/pluggable_device/plugga

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4035 - loss: 1.5435
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6024 - loss: 1.3778
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6521 - loss: 0.9434
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6714 - loss: 1.0318
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6894 - loss: 0.8996
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6843 - loss: 1.0261
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6893 - loss: 0.8930


KeyboardInterrupt: 

In [16]:
#the model has best accuracy when number of feature maps in CL2 is 9
#taking number of epochs as 5 as it is giving best validation accuracy

model = create_model(num_cl2_feature_maps=7)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5, validation_data=(valid_images, valid_labels))

model.save('qn2_cnn.keras')

/Users/adityaviraj/Desktop/DL assgn2/.venv/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 7s 85ms/step - accuracy: 0.2743 - loss: 2.0479 - val_accuracy: 0.4040 - val_loss: 1.4315
Epoch 2/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - accuracy: 0.4613 - loss: 1.3335 - val_accuracy: 0.5380 - val_loss: 1.2201
Epoch 3/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.6032 - loss: 1.0772 - val_accuracy: 0.6040 - val_loss: 1.0718
Epoch 4/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - accuracy: 0.6817 - loss: 0.8594 - val_accuracy: 0.5620 - val_loss: 1.1399
Epoch 5/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - accuracy: 0.7170 - loss: 0.7878 - val_accuracy: 0.5960 - val_loss: 1.0981


In [17]:
model = models.load_model('qn2_cnn.keras')

In [18]:
#predicting the class of train images
train_predictions = model.predict(train_images)
train_predictions = np.argmax(train_predictions, axis=1)
train_loss, train_accuracy = model.evaluate(train_images, train_labels)
train_confusion_matrix = confusion_matrix(train_labels, train_predictions)

#predicting the class of test images
test_predictions = model.predict(test_images)
test_predictions = np.argmax(test_predictions, axis=1)
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
test_confusion_matrix = confusion_matrix(test_labels, test_predictions)


print('Train accuracy:', train_accuracy)
print('Train confusion matrix:')
print(train_confusion_matrix)

print()
print()

print('Test accuracy:', test_accuracy)
print('Test confusion matrix:')
print(test_confusion_matrix)




63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.8574 - loss: 0.4717
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.7211 - loss: 0.7959
Train accuracy: 0.7565000057220459
Train confusion matrix:
[[372   5  17   6   0]
 [ 15 338  32   8   7]
 [ 21  18 350   7   4]
 [ 38 113  42 184  23]
 [ 16  40  53  22 269]]


Test accuracy: 0.6179999709129333
Test confusion matrix:
[[77  6 15  0  2]
 [ 3 81  9  4  3]
 [15  8 74  2  1]
 [ 9 18 23 38 12]
 [ 6 17 30  8 39]]
